In [ ]:
# Define base directory
base_dir = '/kaggle/input/pinku-dataset/Dataset'  # Replace with your actual dataset path

# Create dataset instance
brvo_dataset = BRVODataset(base_dir=base_dir)

# Total number of image-label pairs
print(f"Total pairs in dataset: {len(brvo_dataset)}")

# Example of accessing an item
image, label = brvo_dataset[0]
print(f"Image shape: {image.shape}")
print(f"Label shape: {label.shape}")


In [ ]:
!pip install torch torchvision
!pip install ipywidgets matplotlib


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
import time


# Generator Model
class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()
        
        def conv_block(in_channels, out_channels, kernel_size=4, stride=2, padding=1, batch_norm=True):
            layers = [nn.Conv2d(in_channels, out_channels, kernel_size, stride, padding)]
            if batch_norm:
                layers.append(nn.BatchNorm2d(out_channels))
            layers.append(nn.ReLU(inplace=True))
            return nn.Sequential(*layers)

        def upconv_block(in_channels, out_channels, kernel_size=4, stride=2, padding=1, dropout=False):
            layers = [nn.ConvTranspose2d(in_channels, out_channels, kernel_size, stride, padding)]
            layers.append(nn.BatchNorm2d(out_channels))
            if dropout:
                layers.append(nn.Dropout(0.5))
            layers.append(nn.ReLU(inplace=True))
            return nn.Sequential(*layers)

        self.encoder1 = conv_block(3, 64, batch_norm=False)
        self.encoder2 = conv_block(64, 128)
        self.encoder3 = conv_block(128, 256)
        self.encoder4 = conv_block(256, 512)
        self.encoder5 = conv_block(512, 512)
        self.encoder6 = conv_block(512, 512)
        
        self.decoder1 = upconv_block(512, 512, dropout=True)
        self.decoder2 = upconv_block(1024, 512, dropout=True)
        self.decoder3 = upconv_block(1024, 256)
        self.decoder4 = upconv_block(512, 128)
        self.decoder5 = upconv_block(256, 64)
        self.final_layer = nn.ConvTranspose2d(128, 1, 4, 2, 1)
        self.final_act = nn.Tanh()
        
    def forward(self, x):
        e1 = self.encoder1(x)
        e2 = self.encoder2(e1)
        e3 = self.encoder3(e2)
        e4 = self.encoder4(e3)
        e5 = self.encoder5(e4)
        e6 = self.encoder6(e5)
        
        d1 = self.decoder1(e6)
        d2 = self.decoder2(torch.cat([d1, e5], 1))
        d3 = self.decoder3(torch.cat([d2, e4], 1))
        d4 = self.decoder4(torch.cat([d3, e3], 1))
        d5 = self.decoder5(torch.cat([d4, e2], 1))
        
        output = self.final_layer(torch.cat([d5, e1], 1))
        return self.final_act(output)

# Discriminator Model
class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()
        
        def conv_block(in_channels, out_channels, kernel_size=4, stride=2, padding=1, batch_norm=True):
            layers = [nn.Conv2d(in_channels, out_channels, kernel_size, stride, padding)]
            if batch_norm:
                layers.append(nn.BatchNorm2d(out_channels))
            layers.append(nn.LeakyReLU(0.2, inplace=True))
            return nn.Sequential(*layers)
        
        self.model = nn.Sequential(
            conv_block(4, 64, batch_norm=False),
            conv_block(64, 128),
            conv_block(128, 256),
            conv_block(256, 512, stride=1),
            nn.Conv2d(512, 1, kernel_size=4, stride=1, padding=1),
            nn.Sigmoid()
        )
        
    def forward(self, x):
        return self.model(x)

# Initialize Models
# Initialize Models
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
generator = Generator().to(device)
discriminator = Discriminator().to(device)

# Loss and Optimizers
criterion = nn.BCELoss()
l1_loss = nn.L1Loss()  # L1 loss for generator
lr_gen = 0.0002  # Learning rate for generator
lr_disc = 0.0001  # Lowered learning rate for discriminator

# Further increase lambda L1 weight to push for high contrast
lambda_l1 = 200  # Increased for higher influence
lambda_l2 = 10   # Optional weight for additional L2 loss if needed

optimizer_G = optim.Adam(generator.parameters(), lr=lr_gen, betas=(0.5, 0.999))
optimizer_D = optim.Adam(discriminator.parameters(), lr=lr_disc, betas=(0.5, 0.999))

# Learning rate schedulers
scheduler_G = optim.lr_scheduler.StepLR(optimizer_G, step_size=20, gamma=0.9)
scheduler_D = optim.lr_scheduler.StepLR(optimizer_D, step_size=20, gamma=0.9)

# Training Function with Final Loss Plot
def train(dataloader, num_epochs, lambda_l1, lambda_l2):
    d_losses, g_losses = [], []
    
    print(f"Training for {num_epochs} epochs with Lambda L1 = {lambda_l1} and Lambda L2 = {lambda_l2}")
    
    for epoch in range(num_epochs):
        d_loss_epoch, g_loss_epoch = 0.0, 0.0
        for i, (images, labels) in enumerate(dataloader):
            images, labels = images.to(device), labels.to(device)

            # Train Discriminator
            optimizer_D.zero_grad()
            real_labels = torch.ones(images.size(0), 1, 30, 30).to(device)
            fake_labels = torch.zeros(images.size(0), 1, 30, 30).to(device)

            # Real loss
            real_input = torch.cat([images, labels], dim=1)
            real_output = discriminator(real_input)
            d_real_loss = criterion(real_output, real_labels)

            # Fake loss
            fake_labels_pred = generator(images)
            fake_input = torch.cat([images, fake_labels_pred], dim=1)
            fake_output = discriminator(fake_input.detach())
            d_fake_loss = criterion(fake_output, fake_labels)

            # Total Discriminator Loss
            d_loss = (d_real_loss + d_fake_loss) / 2
            d_loss.backward()
            optimizer_D.step()
            d_loss_epoch += d_loss.item()

            # Train Generator
            optimizer_G.zero_grad()
            fake_output = discriminator(fake_input)
            # GAN loss
            gan_loss = criterion(fake_output, real_labels)
            # Pixel-wise L1 loss
            l1_term = lambda_l1 * l1_loss(fake_labels_pred, labels)
            # Optional L2 (MSE) loss
            l2_term = lambda_l2 * nn.MSELoss()(fake_labels_pred, labels)
            
            # Total Generator Loss
            g_loss = gan_loss + l1_term + l2_term
            g_loss.backward()
            optimizer_G.step()
            g_loss_epoch += g_loss.item()
        
        # Adjust learning rates
        scheduler_G.step()
        scheduler_D.step()

        # Append average epoch loss
        d_losses.append(d_loss_epoch / len(dataloader))
        g_losses.append(g_loss_epoch / len(dataloader))

        # Print progress
        print(f"Epoch [{epoch+1}/{num_epochs}], D Loss: {d_losses[-1]:.4f}, G Loss: {g_losses[-1]:.4f}")
    
    # Plot the final loss graph after training is complete
    plt.figure(figsize=(10, 5))
    plt.plot(d_losses, label="Discriminator Loss", color="red")
    plt.plot(g_losses, label="Generator Loss", color="blue")
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.title('Generator and Discriminator Loss Over Epochs')
    plt.legend()
    plt.grid(True)
    plt.show()

# Example usage:
# Assuming `dataloader` is your DataLoader with (image, label) pairs
train(dataloader, num_epochs=250, lambda_l1=lambda_l1, lambda_l2=lambda_l2)

In [ ]:
# Save the trained generator model
torch.save(generator.state_dict(), "generator100.pth")
print("Generator model saved as 'generator.pth'.")


In [ ]:
import torch
from PIL import Image
from torchvision import transforms
import matplotlib.pyplot as plt

# Load the generator model
def load_generator_model(path="generator.pth"):
    generator = Generator().to(device)
    generator.load_state_dict(torch.load(path, map_location=device))
    generator.eval()  # Set to evaluation mode
    return generator

# Define the transformation for the input image
transform = transforms.Compose([
    transforms.Resize((256, 256)),  # Adjust to your model's input size
    transforms.ToTensor(),
])

# Inference Function
def generate_label(input_image_path, generator_model_path="generator100.pth"):
    # Load the trained generator
    generator = load_generator_model(generator_model_path)
    
    # Load and preprocess the input image
    input_image = Image.open(input_image_path).convert("RGB")
    input_tensor = transform(input_image).unsqueeze(0).to(device)  # Add batch dimension

    # Generate the label
    with torch.no_grad():
        generated_label_tensor = generator(input_tensor)

    # Post-process the output (convert tensor to image)
    generated_label_tensor = (generated_label_tensor.squeeze(0) * 0.5 + 0.5)  # Scale to [0,1]
    generated_label_image = transforms.ToPILImage()(generated_label_tensor.cpu())

    # Display the input image and the generated label side-by-side
    plt.figure(figsize=(10, 5))
    plt.subplot(1, 2, 1)
    plt.title("Input Image")
    plt.imshow(input_image)
    plt.axis("off")

    plt.subplot(1, 2, 2)
    plt.title("Generated Label")
    plt.imshow(generated_label_image, cmap="gray")  # Assuming label is grayscale
    plt.axis("off")

    plt.show()

    # Optionally save the generated label
    generated_label_image.save("generated_label.png")
    print("Generated label saved as 'generated_label.png'.")

# Example usage:
generate_label("/kaggle/input/pinku-dataset/Dataset/S3/s3t2/image/2/2_11460.jpg")  # Replace with the path to your input image


In [ ]:
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
import torch
from skimage.metrics import structural_similarity as ssim

# Function to load and resize images to a fixed size
def load_and_resize_image(image_path, size=(256, 256)):
    image = Image.open(image_path).convert("L")  # Convert to grayscale
    image = image.resize(size, Image.LANCZOS)  # Use LANCZOS for high-quality downsampling
    return np.array(image)

# Paths to your images
original_image_path = "/kaggle/input/pinku-dataset/Dataset/S3/s3t2/label/2/2_11460.jpg"
generated_image_path = "/kaggle/working/generated_label.png"

# Load and resize both images to the same size
original_image = load_and_resize_image(original_image_path)
generated_image = load_and_resize_image(generated_image_path)

# Apply thresholding (binarize) to focus on structure
# Apply binary thresholding to make images purely black and white
threshold_value = 128
original_binary = (original_image > threshold_value).astype(np.uint8)*255 
generated_binary = (generated_image > threshold_value).astype(np.uint8)*255

# Convert images to tensors after thresholding
original_tensor = torch.tensor(original_binary /255)  # Normalize to [0, 1]
generated_tensor = torch.tensor(generated_binary/255)  # Normalize to [0, 1]

# Ensure tensors have the same shape
assert original_tensor.shape == generated_tensor.shape, "The images must have the same dimensions after resizing."

# Re-calculate similarity metrics after thresholding
mse_value = torch.mean((original_tensor - generated_tensor) ** 2).item()
ssim_value = ssim(original_binary, generated_binary, data_range=255)
cosine_similarity_value = torch.nn.functional.cosine_similarity(
    original_tensor.view(-1), generated_tensor.view(-1), dim=0).item()

print(f"Mean Squared Error (MSE): {mse_value:.4f}")
print(f"Structural Similarity Index (SSIM): {ssim_value:.4f}")
print(f"Cosine Similarity: {cosine_similarity_value:.4f}")
